In [ ]:
print(3)

In [1]:
import pandas

In [2]:
import numpy

In [ ]:
import sys
print(sys.executable)

/Users/josealmeida/Desktop/CFM_Data_Challenge/myenv/bin/python


In [3]:
import tensorflow as tf

In [4]:
print(tf.__version__)

2.16.2


In [ ]:
deefe